<h1 id="tocheading">Table of Contents</h1>
<div id="toc"></div>

In [1]:
%%javascript
$.getScript('https://kmahelona.github.io/ipython_notebook_goodies/ipython_notebook_toc.js')

<IPython.core.display.Javascript object>

### 584. Find Customer Referee

```mysql
SELECT name
FROM customer
WHERE referee_id!=2 OR referee_id IS NULL
```

### 585. Investments in 2016

```mysql
SELECT SUM(i.TIV_2016) AS TIV_2016
FROM insurance i
JOIN (
    SELECT lat, lon
    FROM insurance
    GROUP BY lat, lon
    HAVING COUNT(pid)=1
) AS X
ON i.lat=x.lat AND i.lon=x.lon
WHERE TIV_2015 IN (
    SELECT TIV_2015
    FROM insurance
    GROUP BY TIV_2015
    HAVING COUNT(tiv_2015)>1)

/*SELECT SUM(TIV_2016) AS TIV_2016
FROM insurance
WHERE TIV_2015 IN (
    SELECT TIV_2015
    FROM insurance
    GROUP BY TIV_2015
    HAVING COUNT(tiv_2015)>1)
AND CONCAT_WS(',', lat, lon) IN (
    SELECT CONCAT_WS(',', lat, lon)
    FROM insurance
    GROUP BY CONCAT_WS(',', lat, lon)
    HAVING COUNT(pid)=1)*/
```

### 586. Customer Placing the Largest Number of Orders

```mysql
/*SELECT customer_number
FROM orders
GROUP BY customer_number
HAVING COUNT(*) = (
    SELECT MAX(X.n_order)
    FROM (
        SELECT customer_number, COUNT(*) as n_order
        FROM orders
        GROUP BY customer_number) AS X)*/
SELECT customer_number
FROM orders
GROUP BY customer_number
HAVING COUNT(*) >= ALL(
    SELECT COUNT(*) as n_order
    FROM orders
    GROUP BY customer_number)
```

### 595. Big Countries

```mysql
SELECT name, population, area
FROM World
WHERE area>3000000 OR population>25000000
```

### 596. Classes More Than 5 Students

```mysql
SELECT class
FROM courses
GROUP BY class
HAVING COUNT(DISTINCT student)>=5
```

### 597. Friend Requests I: Overall Acceptance Rate

```mysql
SELECT ROUND(IFNULL(COUNT(DISTINCT requester_id, accepter_id)/COUNT(DISTINCT sender_id, send_to_id), 0), 2) AS accept_rate
FROM friend_request, request_accepted

# acceptance rate for each month
/* SELECT IFNULL(n_acc/n_req, 0) as accept_rate
FROM (
    SELECT A.mon, n_req, n_acc
    FROM (SELECT month(request_date) as mon, count(distinct sender_id, send_to_id) as n_req
    FROM friend_request
    GROUP BY month(request_date)) AS A
    JOIN (SELECT month(accept_date) as mon, count(distinct requester_id, accepter_id) as n_acc
    FROM request_accepted
    GROUP BY month(accept_date)) AS B
    ON A.mon=B.mon) AS C*/
```

### 601. Human Traffic of Stadium

```mysql
-- MYSQL
SELECT id, visit_date , people
FROM stadium
WHERE people > 99
AND ( ((id-1 IN (SELECT id FROM stadium WHERE people > 99 ))
    and (id-2 IN (SELECT id FROM stadium WHERE people > 99 )))
OR ((id -1 IN (SELECT id FROM stadium WHERE people > 99 ))
    and (id +1 IN (SELECT id FROM stadium WHERE people > 99 )) )
OR ((id+1 IN (SELECT id FROM stadium WHERE people > 99 ))
    and (id+2 IN (SELECT id FROM stadium WHERE people > 99 ))) )
-- SQL server
         /* WRONG WHERE CONDITIONS */
/*SELECT id, visit_date, people
FROM (
    SELECT id, visit_date, people,
        MIN(people) OVER (ORDER BY id ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) as min_before,
        MIN(people) OVER (ORDER BY id ROWS BETWEEN CURRENT ROW AND 2 FOLLOWING) as min_after,
        ROW_NUMBER() OVER (ORDER BY id) as row,
        COUNT(id) OVER () as n_rows
    FROM stadium ) x
WHERE (min_after>=100 and row<=2)
OR (min_before>=100 and row>(n_rows-2))
OR ((min_before>=100 or min_after>=100)and (row>2 and row<=(n_rows-2)) )*/

SELECT a.id, a.visit_date, a.people
FROM (
    SELECT id, visit_date, people,
    LAG(people, 2) OVER (ORDER BY visit_date) as m2,
    LAG(people, 1) OVER (ORDER BY visit_date) as m1,
    LEAD(people, 1) OVER (ORDER BY visit_date) as p1, 
    LEAD(people, 2) OVER (ORDER BY visit_date) as p2
    FROM stadium) a
WHERE
    (a.people>=100 and a.p1>=100 and a.p2>=100)
or  (a.m2>=100 and a.m1>=100 and a.people>=100)
or  (a.m1>=100 and a.people>=100 and a.p1>=100)
```

### 602. Friend Requests II: Who Has the Most Friends

```mysql
-- MYSQL
SELECT id, num
FROM (
    SELECT id, SUM(cnt) AS num
    FROM (
        SELECT requester_id as id, count(accepter_id) as cnt
        FROM request_accepted a
        GROUP BY requester_id
        UNION ALL
        SELECT accepter_id as id, count(requester_id) as cnt
        FROM request_accepted b
        GROUP BY accepter_id) AS x
    GROUP BY id) AS Y
ORDER BY num DESC
LIMIT 1

-- SQL server
SELECT id, num
FROM (
    SELECT id, num, RANK() OVER (ORDER BY num DESC) AS rnk
    FROM (
        SELECT id, SUM(cnt) AS num
        FROM (
            SELECT requester_id as id, count(accepter_id) as cnt
            FROM request_accepted
            GROUP BY requester_id
            UNION ALL
            SELECT accepter_id as id, count(requester_id) as cnt
            FROM request_accepted
            GROUP BY accepter_id) AS x
    GROUP BY id ) AS y
    ) AS z
WHERE rnk=1
```

### 603. Consecutive Available Seats

```mysql
SELECT DISTINCT a.seat_id AS seat_id
FROM cinema as a
JOIN cinema as b
ON ABS(a.seat_id-b.seat_id)=1 AND (a.free=1 AND b.free=1)
ORDER BY seat_id

/*SELECT DISTINCT a.seat_id
FROM cinema as a
JOIN cinema as b
ON (a.seat_id=b.seat_id-1 AND (a.free=1 AND b.free=1) )
OR (a.seat_id=b.seat_id+1 AND (a.free=1 AND b.free=1) )*/
```

### 607. Sales Person

```mysql
SELECT name
FROM salesperson
WHERE sales_id NOT IN(
    SELECT DISTINCT a.sales_id
    FROM orders AS a
    JOIN company AS b
    ON a.com_id=b.com_id AND b.name='RED')
```

### 608. Tree Node

```mysql
# left join solution
/*SELECT id,
    case when pid=0 then 'Root'
        when pid!=0 and cid!=0 then 'Inner'
        else 'Leaf'
    end AS Type
FROM (
    SELECT a.id as Id, count(a.p_id) as pid, count(b.id) as cid
    FROM tree a
    LEFT JOIN tree b
    ON a.id=b.p_id
    GROUP BY id) X*/

# sub query solution
select id as Id,
    case when p_id is null then 'Root'
         when id in (select p_id from tree) then 'Inner'
         else 'Leaf'
    end as Type
from tree
order by id
```

### 610. Triangle Judgement

```mysql
SELECT x, y, z,
    CASE WHEN x + y > z AND x + z > y AND y + z > x THEN 'Yes'
        ELSE 'No' END AS triangle
FROM triangle
```